# Check for pairs with specific terms (from domain-specific unigram features)

In [4]:
# Load pairs of blog descriptions
import pandas as pd

path = '/data/tumblr_community_identity/dataset114k/matched_reblogs_nonreblogs_dataset114k.csv'
pairs = pd.read_csv(path)
pairs

/projects/tumblr_community_identity/code/conda_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (12,29,38,42,51,54) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,post_id_reblog,activity_time_epoch_reblog,activity_date_reblog,post_title_reblog,post_short_url_reblog,post_slug_reblog,post_type_reblog,post_caption_reblog,post_format_reblog,post_note_count_reblog,...,post_content_reblog_str,reblogged_from_metadata_reblog_str,mentions_reblog_str,post_tags_nonreblog_str,post_content_nonreblog_str,reblogged_from_metadata_nonreblog_str,mentions_nonreblog_str,processed_tumblr_blog_description_follower_reblog,processed_tumblr_blog_description_followee_reblog,processed_tumblr_blog_description_followee_nonreblog
0,180134677524,1542279068,20181115,NaN,https://tmblr.co/ZE4Tsk2dmtGmK,madnessex-shot-by-ricardo-silvestre-for-the,photo,"<p><a href=""http://madnessex.tumblr.com/post/1...",html,231.0,...,"[caption -> <p><a href=""http://madnessex.tumbl...","[parent_post_blog_id -> 117937782, parent_post...",[],"[maleformaz, faggybudz]","[caption -> <p><a href=""http://faggybudz.tumbl...","[parent_post_blog_id -> 106437908, parent_post...",[],the icon/profile pic is a throwback selfie of ...,23|gay|geek|mild kink|europe my stuff validati...,"justin / 28 / phd student / cambridge , uk / a..."
1,178461397088,1537916836,20180926,NaN,https://tmblr.co/ZX4iGy2cD8D1W,grandpasessions-i-feel-my-life-cracking-within,quote,NaN,html,17.0,...,"[format -> html, mentions -> {}, note_count ->...","[parent_post_blog_id -> 76057788, parent_post_...",[],"[facts, factsandchicks, chicks, Minka Kelly, b...",[caption -> <p>Beer is the third most consumed...,[],[],do n't sweat the small stuff,promote your tumblr ! | devil-utopia | | | twi...,creative director and artist based out of nyc ...
2,180607716734,1543464096,20181129,NaN,https://tmblr.co/ZAdeAh2eD3mr_,NaN,photo,NaN,html,8026.0,...,"[caption -> , format -> html, mentions -> {}, ...","[parent_post_blog_id -> 140686598, parent_post...",[],"[Corvette, vette, chevy, chevrolet, cars, auto...","[caption -> <p>1965 Corvette</p>, format -> ht...",[],[],"cute but ... //do n't worry , get a tattoo.//",kübra💫i̇stanbul/18,collection of corvette pictures and videos
3,180056153480,1542078509,20181113,NaN,https://tmblr.co/ZFINix2diBjs8,NaN,photo,NaN,html,5525.0,...,"[caption -> , format -> html, mentions -> {}, ...","[parent_post_blog_id -> 3421480, parent_post_b...",[],[],"[body -> <p><a href=""https://the-eternal-tease...","[parent_post_blog_id -> 200862916, parent_post...",[],queer|oh|27|they/them,i refuse to be a cliché .,"hey baby , when i write , i 'm the hero of my ..."
4,177860778593,1536390010,20180908,NaN,https://tmblr.co/Zd4_9x2bfL1fX,i-have-a-headcanon-that-hermione-insists-her,text,NaN,html,39158.0,...,[body -> I have a headcanon that Hermione insi...,"[parent_post_blog_id -> 19347382, parent_post_...",[],"[sorry if i re-do one or two, ive been meaning...",[caption -> <blockquote><p><small><b>Kommissar...,[],[],i ’ m just a queer nerd who likes to build thi...,obliviate ...,jess . 26 . australian . not spoiler or porn f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110917,180589788455,1543423455,20181128,NaN,https://tmblr.co/ZwkrHa2eB-Nqd,112718,photo,11/27/18,html,41.0,...,"[caption -> 11/27/18, format -> html, mentions...","[parent_post_blog_id -> 366959655, parent_post...",[],[ya bitch me too],"[body -> I just LOVE Australia calum, format -...","[parent_post_blog_id -> 333647828, parent_post...",[],q // nineteen // wrapped around your finger de...,★ we need to talk ★,génèvieve // 19 // i would give my last dying ...
110918,178157783797,1537135868,20180917,NaN,https://tmblr.co/ZW5c-m2bx20hr,NaN,photo,NaN,html,3.0,...,"[caption -> , format -> html, mentions -> {}, ...","[parent_post_blog_id -> 356249823, parent_post...",[],[],"[caption -> <p><a href=""https://kalliokoski.tu...","[parent_post_blog_id -> 433991033, parent_post...",[],"sensibile , pazzamente innamorata di te",instagram : _tought.face_,sw:57kg | cw : 58kg | gw:45kg hi im fatty bitch
110919,180618703114,1543498913,20181129,تفاصيلك وأنا النّساي أذكرها !,https://tmblr.c

In [29]:
# desc_cols = [col for col in pairs.columns if 'description' in col and not 'processed' in col]
desc_cols = [col for col in pairs.columns if 'description' in col and 'processed' in col]
desc_cols

['processed_tumblr_blog_description_follower_reblog',
 'processed_tumblr_blog_description_followee_reblog',
 'processed_tumblr_blog_description_followee_nonreblog']

In [9]:
id_cols = [col for col in pairs.columns if 'tumblog_id' in col]
id_cols

['tumblog_id_follower_reblog',
 'tumblog_id_followee_reblog',
 'tumblog_id_followee_nonreblog']

In [8]:
# Load communities
commpath = '/data/tumblr_community_identity/dataset114k/louvain_communities.txt'
with open(commpath) as f:
    comms = [[int(tumblog_id) for tumblog_id in comm.split()] for comm in f.read().splitlines()]
print(len(comms))
comms[0]

82


[428420260,
 309639953,
 389871600,
 417529847,
 166797353,
 361747941,
 299317489,
 326035904,
 334537238,
 423340917,
 417422437,
 422384449,
 67391670,
 317236249,
 302511340,
 370318017,
 370610877,
 382924416,
 343931759,
 384167619,
 258268338,
 441501897,
 143375479,
 377830815,
 388114130,
 375514918,
 97439529,
 434513951,
 430747586,
 329069078,
 246319825,
 335540728,
 215506790,
 39658196,
 354407284,
 432428040,
 432045214,
 444086477,
 429081590,
 430878984,
 241994668,
 423230490,
 432971807,
 291438935,
 390164044,
 446139132,
 439073411,
 422992881,
 423360934,
 376747767,
 442717932,
 427159933,
 382039261,
 427199088,
 291931032,
 368905655,
 391809892,
 401324024,
 209837250,
 422897527,
 412530844,
 402119381,
 425094001,
 429716016,
 423361525,
 401832738,
 421855663,
 424177364,
 422844683,
 249653570,
 442399052,
 255915805,
 323642466,
 312083964,
 108204201,
 309646941,
 427890038,
 409498007,
 424222849,
 423211254,
 398492047,
 438260800,
 363146282,
 427856

In [42]:
# Investigate
comm = 49
terms = ['blog', 'dream']

commlines = pairs[pairs['tumblog_id_follower_reblog'].isin(comms[comm-1]) & pairs['tumblog_id_followee_reblog'].isin(comms[comm-1]) & \
                 pairs['tumblog_id_followee_nonreblog'].isin(comms[comm-1])]
commlines

import re
import pdb
pd.set_option('display.max_colwidth', None)

def terms_in_descs(descs):
    """ Search for the search terms being in the descriptions """
    follower, followee_reblog, followee_nonreblog = descs
    if not isinstance(follower, str):
        return False
    pat1 = '\\b{}\\b'.format(terms[0])
    pat2 = '\\b{}\\b'.format(terms[1])
    if re.search(pat1, follower.lower()):
        if re.search(pat2, followee_reblog) or re.search(pat2, followee_nonreblog):
            return True
    elif re.search(pat2, follower.lower()):
        if re.search(pat1, followee_reblog) or re.search(pat1, followee_nonreblog):
            return True
    return False

paired_descs = list(zip(*[commlines[colname] for colname in desc_cols]))
matches = commlines.loc[[terms_in_descs(descs) for descs in paired_descs], desc_cols]
matches.sample(min(30, len(matches)))

,processed_tumblr_blog_description_follower_reblog,processed_tumblr_blog_description_followee_reblog,processed_tumblr_blog_description_followee_nonreblog
97569,"thanks to tumblr , my inner girl , summer , has become more and more prominent and is trying to break free . for now , being her remains a dream , but the veil between keeping her locked away and allowing her to become reality is becoming quite thin indeed . it 's only a matter of time before realities switch and summer becomes the dominant personality . i hope to share that journey here . stay beautiful !",formerly sissysandstuds . just a place to hold my dearest thoughts on cute clothes and embracing my girly side . some things are nsfw so if you are not of the legal age for where you live please just dont explore my blog . captions will still be marked # sas . reblog dont post .,"strictly 18+ only | nsfw read more about me , my blog & my rules , particularly before contacting me : http : //tmblr.co/zfpbvl1igvcr5"
21803,18+ the profile pic is me and i am an aspiring sissy slut from oklahoma . i live to serve all men . it is my dream to one day be owned by a man and to be his personal slave fuck toy . feel free to message me on here or you can kik me @ bottomboi1997 .,"hi , i 'm a 24 year old virgin that wants and fantasizes about being sissified , `` forced '' to suck cock , and fucked hard up my ass where i 'm just left face down with cum oozing out of me . my captions are all tagged . be free to message me and if you 're from chicago do n't hesitate to contact me .","this blog is nsfw and will have pictures containing adult themes . if you are not legally allowed to view them , or are offended by them , please leave immediately . if you find a picture that you would like removed please message me ."
75115,"must be 18yrs or older to view this blog , no time for kids . i am a black male 45 , straight i have been a dom for over 20 years . i like to fuck all kinds of women , but mostly white . i have played with couples before . some hubs like to watch , some like to join , some just like to clean up . if you are in mississippi or tennessee let me know . if you have any questions just ask . kik bbcinstructor","we are a couple from texas . nsfw stuff i like , and some pics of us . 18 up only ! ! ! i do not own most pics on here , just reblogs of stuff i like , if you want one removed please let me know .","your sweetest dream and worst nightmare wrapped in black skin . my hair isn ’ t the only thing that is kinky . i ’ m a 24 year old bisexual cuckcake/unicorn who is interested in black female supremacy , swinging , bdsm and a myriad of other sinful delights . i am also a proud sex worker ( erotic dancer and dominatrix ) . leave your inhibitions at the door as this blog is nsfw . all of my writings are tagged # cuckcakewrites . all images of me are tagged # ypcc . my stories focus on sexually liberated black women , there will be zero fetishization in this space . i plan to showcase a more uplifting side of raceplay for participants of african descent . pro black does not mean anti-white ."
99222,making my way through this thing called life . finding new pleasures . find my sister blog @ dirtyprettier,i would postpone my dream for you .,under 18 leave . 36 happily married bi curious male looking for new kinky ideas and sharing some pictures . kik midwest7981
51201,moving my nsfw content from tumblr world and stayinghere for a while to connect treasured people here . request you to check out my mewe .. https : --mewe-com-i-nakedlifeluv or snapchat by the name 'bert.love ' twitter @ darknightsecre1 🔞 🔞 .. dream life .. 😉 secret account 😇 not interested in dating sites 🔞 any dating sites following me will be blocked ... straight male 💏💏,this is my page dedicated to pictures that interest me and turn me on .,"nsfw , 18+ only ! we are a real couple that is in the lifestyle . we are in the nw louisiana/e texas region . 38f/30m . very kinky and always horny . we are here to ha

# Examine features in training and test sets for different datasets

In [18]:
# Load features
import os
import pickle
from tqdm.notebook import tqdm

zero_test_features = {}
total_features = {}
runs = [
    'baseline_randomtest+exp1+exp2_all_lr'
    'dataset114k+exp1+exp2_all_lr',
]

for run in runs:
    print(run)
    run_fname = run + '_features.pkl'
    features_dirpath = '/projects/websci2020_tumblr_identity/output/features'
    features_fpath = os.path.join(features_dirpath, run_fname)
    with open(features_fpath, 'rb') as f:
        X_train, y_train, X_test, y_test = pickle.load(f)
    print(X_train.shape)
    print(X_test.shape)
    print()

    # Count features seen in training set that are never seen in the test set
    X_train_arr = X_train.A
    empty_features_train = []
    for col in tqdm(range(X_train.shape[1])):
        if not X_train_arr[:,col].any():
            empty_features_train.append(col)
    print(len(empty_features_train)) # why are there any?
    total_features[run] = len(X_train.shape[1])
    print(X_train.shape[1])
    print()

    X_test_arr = X_test.A
    empty_features_test = []
    for col in tqdm(range(X_test.shape[1])):
        if not X_test_arr[:,col].any():
            empty_features_test.append(col)
    print(len(empty_features_test))
    zero_test_features[run] = len(set(empty_features_test) - set(empty_features_train))
    print(zero_test_features[run])
    print()

dataset114k+exp1+exp2_all_lr
(101327, 27551)
(11259, 27551)




218
27551




18163
17955
baseline_randomtest+exp1+exp2_all_lr
(641403, 17744)
(71267, 17744)




8
17744




5683
5675


In [20]:
total_features = {'baseline_randomtest+exp1+exp2_all_lr': 17744,
    'dataset114k+exp1+exp2_all_lr': 27551}

In [21]:
proportions_empty_features = {key: zero_test_features[key]/total_features[key] for key in zero_test_features}
proportions_empty_features

{'dataset114k+exp1+exp2_all_lr': 0.6517004827410984,
 'baseline_randomtest+exp1+exp2_all_lr': 0.3198264201983769}